In [2]:
import pandas as pd

## Importing data

In [3]:
url = "https://docs.google.com/spreadsheets/d/17o2TJJ3_pmrFsFNIdhxyPW3PFO0zyksoSVbrWzrDJoU/export?format=xlsx"
dataset = pd.ExcelFile(url, engine='openpyxl')

sheets = []
for sheet in dataset.sheet_names:
    df = dataset.parse(sheet)
    df["group"] = sheet #adding a column to track from which group is the data
    sheets.append(df)

lego_df = pd.concat(sheets, ignore_index=True)

In [4]:
dataset.sheet_names

['group1', 'group 2', 'group_3', 'group 4', 'group_5']

In [5]:
lego_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               0 non-null      float64
 1   color            204 non-null    object 
 2   is duplo?        204 non-null    object 
 3   size type        204 non-null    object 
 4   base shape       204 non-null    object 
 5   base dimensions  204 non-null    object 
 6   number of studs  204 non-null    int64  
 7   has slope?       204 non-null    object 
 8   slope degree     168 non-null    float64
 9   in stock         204 non-null    object 
 10  group            204 non-null    object 
 11  transparent      40 non-null     float64
dtypes: float64(3), int64(1), object(8)
memory usage: 19.3+ KB


## Data inspection

In [6]:
df = lego_df.copy()
df.sample(10)

,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock,group,transparent
91,NaN,pear,no,plate,rectangle,4 X 1,0,no,0.0,1,group_3,NaN
87,NaN,white,yes,plate,rectangle,8 X 2,16,no,0.0,1,group_3,NaN
122,NaN,neon blue,no,plate,circle,1 X 1,0,no,0.0,1,group_3,NaN
39,NaN,Bright Pink,False,Plate,Circle,1 x 1,1,False,NaN,1,group1,NaN
100,NaN,sky blue,no,brick,rectangle,2 X 1,1,yes,45.0,1,group_3,NaN
105,NaN,bright yellow,no,plate,square,1 X 1,1,no,0.0,1,group_3,NaN
197,NaN,Transparent yellow,False,Plate,Rectangle,1*2,2,False,0.0,1,group_5,NaN
1,NaN,Green,True,Brick,Rectangle,2 x 4,8,False,NaN,1,group1,NaN
202,NaN,Transparent sky blue,False,Plate,Round,1*1,0,False,0.0,1,group_5,NaN
110,NaN,black,no,brick,square,2 X 1,2,yes,45.0,1,group_3,NaN


In [21]:
df.loc[df["group"] == "group1", "base dimensions"].unique()

array(['2x4', '2x2', '2x8', '1x4', '1x3', '2x6', '1x2', '4x6', '1x1',
       '2x3', '0x0', '2x4+2x2'], dtype=object)

In [20]:
df.loc[df["group"] == "group 2", "base dimensions"].unique()

array(['2x4', '2x8', '2x2', '2x6', '2x3', '1x3', '1x4', '1x2', '1x1',
       '4x6'], dtype=object)

In [41]:
df.loc[df["group"] == "group_3", "base dimensions"].unique()

array(['2x4', '2x2', '2x8', '2x6', '1x4', '1x2', '2x3', '4x6', '1x1',
       '4x4', '1x3'], dtype=object)

In [18]:
df.loc[df["group"] == "group 4", "base dimensions"].unique()

array(['2x4', '2x8', '2x2', '2x6', '2x3', '1x2', '1x4', '1x3', '1x1',
       '4x4', '4x6'], dtype=object)

In [17]:
df.loc[df["group"] == "group_5", "base dimensions"].unique()

array(['2x4', '2x2', '2x8', '4x6', '1x4', '2x6', '1x3', '4x4', '1x2',
       '1x1'], dtype=object)

## Data cleaning

### cleaning base dimensions

In [12]:
#removing white spaces
df["base dimensions"] = df["base dimensions"].str.replace(" ", "")

In [15]:
#replacing "*" by "x"
df["base dimensions"] = df["base dimensions"].str.replace("*", "x")

In [16]:
#to lowercase
df["base dimensions"] = df["base dimensions"].str.lower()

In [22]:
#changing 0x0
#df[df["base dimensions"] == "0x0"]
df.loc[df["base dimensions"] == "0x0", "base dimensions"] = "1x1"

In [23]:
#changing 2x4+2x2
#df[df["base dimensions"] == "2x4+2x2"]
df.loc[df["base dimensions"] == "2x4+2x2", "base dimensions"] = "4x4"

In [34]:
x = "gustavo"

In [39]:
x[::-1]

'ovatsug'

In [40]:
# putting the dimensions in the same format
for index, row in df.iterrows():
    base = row["base dimensions"]
    if int(base[0]) > int(base[-1]):
        df.loc[index,"base dimensions"] = base[::-1]
    else:
        continue

### Cleaning colors

In [42]:
df.loc[df["group"] == "group1", "color"].unique()

array(['Dark Blue', 'Green', 'Coral', 'Red', 'White', 'Light Green',
       'Orange', 'Yellow', 'Brown', 'Purple', 'Lilac', 'Orchid',
       'Dark Grey', 'Light Grey', 'Pink', 'Dark Green', 'Cream', 'Black',
       'Red Brown', 'Neon Yellow', 'Cyan', 'Denim Blue', 'Ice Blue',
       'Capri', 'Shiny Green', 'Olive Green', 'Beige', 'Bright Pink',
       'Transparent'], dtype=object)

In [43]:
df.loc[df["group"] == "group 2", "color"].unique()

array(['lightgreen', 'lightbrown', 'lightpink', 'red', 'yellow',
       'darkblue', 'darkgreen', 'darkpink', 'orange', 'darkpurple',
       'lightpurple', 'white', 'grey', 'mudbrown', 'black', 'turquoise',
       'lightblue', 'neon orange', 'neon dark yellow',
       'neon light yellow', 'darkbrown'], dtype=object)

In [44]:
df.loc[df["group"] == "group_3", "color"].unique()

array(['red', 'bright green', 'pink', 'peach', 'blue', 'white', 'pear',
       'dark green ', 'dark green', 'turquoise', 'sky blue', 'yellow',
       'bright yellow', 'black', 'navy blue', 'grey', 'purple', 'orange',
       'maroon', 'cream', 'khaki', 'neon green', 'neon yellow',
       'neon blue'], dtype=object)

In [45]:
df.loc[df["group"] == "group 4", "color"].unique()

array(['coral', 'lime green', 'red', 'black', 'orange', 'blue', 'green',
       'purple', 'grey', 'brick red', 'cyan', 'white', 'pink',
       'navy blue', 'baby blue', 'yellow', 'cafe', 'neon yellow',
       'dark green', 'hot magent'], dtype=object)

In [46]:
df.loc[df["group"] == "group_5", "color"].unique()

array(['Green ', 'Red', 'Blue', 'Purple', 'Yellow', 'Light Green',
       'White', 'Black', 'Grey', 'Beige', 'Sky Blue', 'Deep Blue', 'Pink',
       'Green', 'Brown', 'Sky blue', 'Dark Blue', 'Transparent yellow',
       'Transparent orange', 'Blck', 'Transparent sky blue'], dtype=object)

In [47]:
df["color"].unique()

array(['Dark Blue', 'Green', 'Coral', 'Red', 'White', 'Light Green',
       'Orange', 'Yellow', 'Brown', 'Purple', 'Lilac', 'Orchid',
       'Dark Grey', 'Light Grey', 'Pink', 'Dark Green', 'Cream', 'Black',
       'Red Brown', 'Neon Yellow', 'Cyan', 'Denim Blue', 'Ice Blue',
       'Capri', 'Shiny Green', 'Olive Green', 'Beige', 'Bright Pink',
       'Transparent', 'lightgreen', 'lightbrown', 'lightpink', 'red',
       'yellow', 'darkblue', 'darkgreen', 'darkpink', 'orange',
       'darkpurple', 'lightpurple', 'white', 'grey', 'mudbrown', 'black',
       'turquoise', 'lightblue', 'neon orange', 'neon dark yellow',
       'neon light yellow', 'darkbrown', 'bright green', 'pink', 'peach',
       'blue', 'pear', 'dark green ', 'dark green', 'sky blue',
       'bright yellow', 'navy blue', 'purple', 'maroon', 'cream', 'khaki',
       'neon green', 'neon yellow', 'neon blue', 'coral', 'lime green',
       'green', 'brick red', 'cyan', 'baby blue', 'cafe', 'hot magent',
       'Green ', 'Bl

In [ ]:
#creating a dictionary to map all the different variations of the colors
color_groups = {
    "black": [
        "black", "blck"
    ],
    "white": [
        "white"
    ],
    "grey": [
        "grey", "gray", "light grey", "dark grey"
    ],
    "blue": [
        "blue", "dark blue", "deep blue", "navy blue", "denim blue", 
        "sky blue", "skyblue", "ice blue", "baby blue", "capri", 
        "lightblue", "transparent sky blue", "transparent skyblue", "neon blue"
    ],
    "green": [
        "green", "light green", "dark green", "dark green ", "green ",
        "olive green", "lime green", "neon green", "shiny green", 
        "bright green", "pear", "khaki", "lightgreen"
    ],
    "yellow": [
        "yellow", "neon yellow", "neon light yellow", "neon dark yellow",
        "bright yellow", "transparent yellow"
    ],
    "orange": [
        "orange", "neon orange", "transparent orange"
    ],
    "red": [
        "red", "brick red", "red brown", "maroon", "coral", "peach"
    ],
    "pink": [
        "pink", "darkpink", "lightpink", "bright pink", "hot magent", "dark pink"
    ],
    "purple": [
        "purple", "dark purple", "light purple", "lilac", "orchid"
    ],
    "brown": [
        "brown", "lightbrown", "darkbrown", "mudbrown", "cafe"
    ],
    "beige": [
        "beige", "cream"
    ],
    "cyan": [
        "cyan", "turquoise"
    ],
    "transparent": [
        "transparent"
    ]
}

In [48]:
#creating function that will be applied to each color name/label
def normalize_color(label):
    label = label.strip().lower()
    for group, variants in color_groups.items():
        if label in variants:
            return group
    return "other"

In [ ]:
#creating a new columns with the normalized names
df["color_group"] = df["color"].apply(normalize_color)

In [ ]:
df[df["color_group"]=="green"]

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               0 non-null      float64
 1   color            204 non-null    object 
 2   is duplo?        204 non-null    object 
 3   size type        204 non-null    object 
 4   base shape       204 non-null    object 
 5   base dimensions  204 non-null    object 
 6   number of studs  204 non-null    int64  
 7   has slope?       204 non-null    object 
 8   slope degree     168 non-null    float64
 9   in stock         204 non-null    object 
 10  group            204 non-null    object 
 11  transparent      40 non-null     float64
dtypes: float64(3), int64(1), object(8)
memory usage: 19.3+ KB
